# 정령 시연 시나리오 (던전)  
## 1층 던전 입장  
- "인벤토리에 아이템이 뭐 있어?" (STT 사용 모습 보여줌)
- "xxxx 사용해" (장비무기 사용 상호작용)
- "던전의 밝기좀 켜줘" (던전 밝기 조절 on)

## 던전 플레이 도중
- "던전의 밝기좀 꺼줘" (던전 밝기 조절 off)

## 보스몬스터 조우
- "보스몬스터의 전투 패턴을 알려줘" 

In [1]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))
from agents.fairy.dungeon.fairy_dungeon_agent import (
    graph_builder as dungeon_graph_builder,
)

from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from core.game_dto.StatData import StatData
from agents.fairy.fairy_state import DungeonPlayerState

from agents.fairy.interaction.fairy_interaction_agent import (
    graph_builder as interaction_builder,
)
from langchain.messages import HumanMessage
from agents.fairy.memory_messages import get_fairy_messages_dungeon
from core.common import get_inventory_items, get_skills

dungeon_graph = dungeon_graph_builder.compile()
interaction_graph = interaction_builder.compile()

async def invoke_graphs(
    question: str,
    target_monster_ids=[], 
    weapon=MockFactory.create_weapon(),
    currrRoom_id = 1,
):
    inventory_ids = [20, 40, 46, 67, 110]
    stats = StatData(
        strength=10,
        dexterity=10,
        intelligence=10
    )
    inventory = get_inventory_items(inventory_ids, stats)    
    skill_ids = [0, 1]
    skills = get_skills(skill_ids)

    player = DungeonPlayerState(
        playerId="BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
        heroineId=1,
        currRoomId=currrRoom_id,
        difficulty=0,
        stats=stats,
        skills=skills,
        weapon=weapon,
        inventory=inventory,
    )

    memories = get_fairy_messages_dungeon(
        player_id=player.playerId, heroine_id=player.heroineId, limit=4
    )

    inputs = {
        "messages": memories + [add_human_message(content=question)],
        "dungenon_player": player,
        "target_monster_ids": target_monster_ids,
    }

    dungeon_talk = await dungeon_graph.ainvoke(inputs)

    interaction_talk = interaction_graph.invoke(
        {
            "inventory": inventory_ids,
            "messages": memories + [HumanMessage(question)],
            "weapon": player.weapon,
            "stats": player.stats,
        }
    )
    print("기억::", memories)
    print("질문::", dungeon_talk["messages"][-2])
    print("응답::", dungeon_talk["messages"][-1])
    print(
        "상호 작용:",
        {
            "useItemId": interaction_talk["useItemId"],
            "roomLight": interaction_talk["roomLight"],
            "temp_use_item_id": interaction_talk["temp_use_item_id"],
            "is_item_use": interaction_talk["is_item_use"],
            "latency_create_temp_use_item_id": interaction_talk["latency_create_temp_use_item_id"]
        },
    )


/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 242445.32it/s]
You're using a XLMRobertaTokenizerFast

In [2]:
await invoke_graphs("어떻게 깨?",target_monster_ids=[5])

총 질문 <Used Abilities>
['EVENT_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarit

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


invalid literal for int() with base 10: 'None'
기억:: [HumanMessage(content='레어 쌍검 써줘!', additional_kwargs={'created_at': '2026-01-07T22:05:33.305243+09:00'}, response_metadata={}, id='a0722aff-92e9-46ed-b0a7-1db2c3c324a1'), AIMessage(content='알겠어! 레어 쌍검으로 장착했어.', additional_kwargs={'created_at': '2026-01-07T22:05:33.350346+09:00', 'intent_types': ['INTERACTION_HANDLER']}, response_metadata={}, id='7c8e9f69-a6b1-403b-a4ab-e2bb271d635c'), HumanMessage(content='내 인벤토리 레어 쌍검은 이미 장착중인데?', additional_kwargs={'created_at': '2026-01-07T22:05:35.327340+09:00'}, response_metadata={}, id='d9af9471-aa64-4e1e-99f0-bb9cff4091da'), AIMessage(content='응? 레어 쌍검은 이미 장착 중이야. 바꿀 필요 없어!', additional_kwargs={'created_at': '2026-01-07T22:05:35.372932+09:00', 'intent_types': ['INTERACTION_HANDLER']}, response_metadata={}, id='a41281e7-4b45-47c0-a445-1f7cbd305b8d')]
질문:: content='어떻게 깨?' additional_kwargs={'created_at': '2026-01-07T22:08:02.928807'} response_metadata={} id='4343d613-2359-4756-ba11-89e3696f3091'

In [3]:
await invoke_graphs("체력은 어떻게 채워?",target_monster_ids=[5])

총 질문 <Used Abilities>
['USAGE_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarit

In [4]:
await invoke_graphs("저기에 빛나는건 뭐야?")

총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    

In [5]:
await invoke_graphs("상호작용은 어떻게 하는데?")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [6]:
await invoke_graphs("인벤토리에 가장 쎈무기 두개 알려줘")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [7]:
#FALSE
await invoke_graphs("둘중에 가장 약한걸로 껴줘")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [8]:
from core.game_dto.z_muck_factory import MockFactory
await invoke_graphs('무기 교체 해줘!', weapon= MockFactory.create_yakk_weapon())

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 20,
    "weaponType": 3,
    "weaponName": "레어 드래곤슬레이어",
    "rarity": 

In [9]:
await invoke_graphs('레전드 바이킹의 망치로 교체해줘', target_monster_ids=[1001])

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
  

In [10]:
await invoke_graphs('던전의 밝기좀 켜줘')

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [11]:
await invoke_graphs("인벤토리에 뭐 있어?")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [12]:
await invoke_graphs("중량의 반지 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [13]:
#8
await invoke_graphs("고양이의 털 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [14]:
await invoke_graphs("아이템 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [15]:
await invoke_graphs("숏쇼드랑 드래곤 슬레이어 데미지 몇인데")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [16]:
#FALSE
await invoke_graphs("그럼 둘중 약한걸로 써줘", weapon= MockFactory.create_power_weapon())

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 67,
    "weaponType": 4,
    "weaponName": "레전드 바이킹의 망치",


In [17]:
#FALSE
await invoke_graphs("고급 드워프의 망치 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [18]:
#13
await invoke_graphs("레어 쌍검과 레어 드래곤슬레이어는 다른거잖아", weapon= MockFactory.create_power_weapon())

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 67,
    "weaponType": 4,
    "weaponName": "레전드 바이킹의 망치",
    "rarity": 3,
    "a

In [19]:
await invoke_graphs("레어 드래곤 슬레이어 장착해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [20]:
#FALSE
await invoke_graphs("레어 쌍검 써줘!")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [21]:
#15
await invoke_graphs("내 인벤토리 레어 쌍검은 이미 장착중인데?")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [22]:
#16
await invoke_graphs("스킬 사용법이 뭐야?")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [23]:
#17
await invoke_graphs("저 보스몬스터는 뭐야?", target_monster_ids=[1000])

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "att

In [24]:
#18
await invoke_graphs("무기 스왑 해줘", target_monster_ids=[1000])

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
  

In [25]:
#19
await invoke_graphs("인벤토리에 악세사리 뭐있어?", target_monster_ids=[1000])

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
  

In [26]:
#20
await invoke_graphs("현재 방에 몬스터 얼마나 있어?", currrRoom_id=1)

총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    

In [27]:

#FALSE
await invoke_graphs("저거 뭐야?", target_monster_ids=[1000])

총 질문 <Used Abilities>
['SMALLTALK', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity"

In [28]:
#22
await invoke_graphs("사용키좀 알려줘", target_monster_ids=[1000])

총 질문 <Used Abilities>
['USAGE_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarit

In [29]:
#23
await invoke_graphs("이벤트 어떻게 꺠?", target_monster_ids=[1000])

총 질문 <Used Abilities>
['EVENT_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarit

In [30]:
#24
await invoke_graphs("내 인벤토리에 쓸만한 장비 2개만 말해봐")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [31]:
#25
await invoke_graphs("둘중 쌘거 껴줘")

총 질문 <Used Abilities>
['INTERACTION_HANDLER', 'SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "r

In [32]:
#26
await invoke_graphs("안녕 반가워!")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [33]:
#27
await invoke_graphs("방금 내가 뭐라고 했어?")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [34]:
#28
await invoke_graphs("스킬 사용해")

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
  

In [35]:
#29
await invoke_graphs("일반 대검과 일반 쌍검의 데미지가 어떻게 돼?")

총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attac

In [36]:
#30
await invoke_graphs("배고프네 먹을것 추천해줘")

총 질문 <Used Abilities>
['SMALLTALK']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-UF399PPBM3-BA9F71BC44D7ACD5707415AB64E2197E",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 10,
    "dexterity": 10,
    "intelligence": 10,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skills": [
    {
      "skillId": 0,
      "skillName": "수리검",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "원거리, 빠른 공격속도"
    },
    {
      "skillId": 1,
      "skillName": "사자베기",
      "skillDescription": null,
      "skillCoolTime": 1.0,
      "IsWeaponSkill": false,
      "typeOrId": null,
      "keyword": "근거리, 강한 한방, 넉백"
    }
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackP

In [37]:
#31
await invoke_graphs("내가 방금 질문한 내용이 뭐였지?")

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=FairyDungeonIntentOutput>["intents": ["SMALLTALK"]]</function>'}}

In [ ]:
#32
await invoke_graphs("저 몬스터 뭐야?")

In [ ]:
#33
await invoke_graphs("저기 있는 적 이름이 뭐야",target_monster_ids=[5])

In [ ]:
#34
await invoke_graphs("스켈레톤 왜 이렇게 단단해",target_monster_ids=[0])

In [ ]:
#35
await invoke_graphs("저 몬스터 체력 얼마나 돼",target_monster_ids=[1])

In [ ]:

#36
await invoke_graphs("공격력이 센 편이야?",target_monster_ids=[4])

In [ ]:
#37
await invoke_graphs("움직임이 느린 이유가 있어?")

In [ ]:
#38
await invoke_graphs("저거 보스야?",target_monster_ids=[3])

In [ ]:
#39
await invoke_graphs("보스면 패턴이 뭐야?",target_monster_ids=[1000])

In [ ]:
#40
await invoke_graphs("약한 공격 방식이 있어?",target_monster_ids=[2])


In [ ]:
#41
await invoke_graphs("저 몬스터는 근접이 위험해?", target_monster_ids=[1])

In [ ]:
#42
await invoke_graphs("저거 공격 범위 넓어 보여?", target_monster_ids=[1001])

In [ ]:
#43
await invoke_graphs("저 몬스터 스태거 잘 걸려?", target_monster_ids=[3])

In [ ]:
#44
await invoke_graphs("경험치 많이 줘?",target_monster_ids=[1000])

In [ ]:
#45
await invoke_graphs("언데드 같은데 맞아?", target_monster_ids=[0])

In [ ]:
#46
await invoke_graphs("저 적이 다른 몬스터보다 센 이유가 뭐야?", target_monster_ids=[1001])

In [ ]:
#47
await invoke_graphs("여러 마리 같이 나오면 위험해?", target_monster_ids=[5])

In [ ]:
#48
await invoke_graphs("지금 내 상태로 잡을 수 있어?", target_monster_ids=[4])

In [ ]:
#49
await invoke_graphs("공격 속도가 높은 편이야?", target_monster_ids=[4])

In [ ]:
#50
await invoke_graphs("저거랑 전에 본 몬스터랑 뭐가 달라?", target_monster_ids=[3])

In [ ]:
#51
await invoke_graphs("전투는 어떻게 하는 거야?")

In [ ]:
#52
await invoke_graphs("스킬은 언제 쓰는 게 좋아")

In [ ]:
#53
await invoke_graphs("이 게임에서 던전은 왜 도는 거야")

In [ ]:
#54
await invoke_graphs("여기서 나가려면 어떻게 해야 돼")

In [ ]:
#55
await invoke_graphs("아이템은 어떻게 사용하는 거야")

In [ ]:
#56
await invoke_graphs("무기 바꾸면 뭐가 달라져")

In [ ]:
#57
await invoke_graphs("공격 속도는 전투에 어떤 영향을 줘")

In [ ]:
#58
await invoke_graphs("스태거가 정확히 뭐야")

In [ ]:
#59
await invoke_graphs("지금 같은 상황에서 조심해야 할 게 있어")

In [ ]:
#60
await invoke_graphs("이 게임에서 중요한 스탯이 뭐야")

In [ ]:
#61
await invoke_graphs("던전 클리어하면 뭐가 좋아")

In [ ]:
#62
await invoke_graphs("보스 전투 전에 준비해야 할 게 있어")

In [ ]:
#63
await invoke_graphs("이 상황에서 안전하게 싸우는 방법이 뭐야")

In [ ]:
#64
#FALSE
await invoke_graphs("장비 등급 차이가 큰 편이야")

In [ ]:
#65
await invoke_graphs("난이도가 높아지면 뭐가 달라져")

In [ ]:
#66
await invoke_graphs("이 던전 구조는 어떤 식이야")

In [ ]:
#67
await invoke_graphs("전투 중에 위치 잡는 게 중요해")

In [ ]:
#68
await invoke_graphs("공격만 계속 해도 되는 거야")

In [ ]:
#69
await invoke_graphs("이 상태로 계속 진행해도 괜찮아")


In [ ]:
#70
await invoke_graphs("초반에 어떤 식으로 싸우는 게 좋아")


In [ ]:
#71
await invoke_graphs("저거 몬스터 뭐야 좀 위험한거야")

In [ ]:
#72
await invoke_graphs("불 좀 켜고 저기 적 정보 알려줘")

In [ ]:
#73
await invoke_graphs("다음 방 어디야 위험해 보여")

In [ ]:
#74
await invoke_graphs("이거 이벤트야 그냥 지나가도 돼",currrRoom_id=2)

In [ ]:
#75
await invoke_graphs("저 몬스터 약점 뭐야 그리고 지금 싸워도 돼",target_monster_ids=[5])


In [ ]:
#76
#FALSE
await invoke_graphs("이 장치 뭐하는거야 건드리면 안되나",currrRoom_id=2)


In [ ]:
#77
await invoke_graphs("나 지금 상태로 보스 잡을수있어")


In [ ]:
#78
#FALSE (인터렉션은 불꺼짐)
await invoke_graphs("불 꺼도 돼? 근데 저기 뭐 나올거같은데")

In [ ]:
#79
#FALSE (인터렉션은 사용)
await invoke_graphs("아이템 쓰고 싶은데 지금 써도 괜찮아")

In [ ]:
#80
await invoke_graphs("여기서 뭐부터 해야돼 저거 몬스터부터야", currrRoom_id=2)


In [ ]:
#81
await invoke_graphs("이 방 이벤트면 실패하면 아픈거야", currrRoom_id=2)


In [ ]:
#82
await invoke_graphs("다음 길 가기전에 조심할거 있어")

In [ ]:
#83
await invoke_graphs("저 몬스터 공격 빠른편이야 위험해")

In [ ]:
#84
await invoke_graphs("이거 그냥 싸우는거 맞지 다른거 없어")

In [ ]:
#85
await invoke_graphs("무기 바꾸는게 나아 지금 몬스터 보니까")

In [ ]:
#86
#FALSE (인터랙션은 불을 끔)
await invoke_graphs("여기 불 안켜도 진행은 되지")

In [ ]:
#87
await invoke_graphs("이 상황에서 싸우는게 맞는지 모르겠어")

In [ ]:
#88
await invoke_graphs("저 장치 먼저 만지면 전투 나와",currrRoom_id=2)

In [ ]:
#89
await invoke_graphs("지금 던전 계속 가도 되는거야")

In [ ]:
#90
await invoke_graphs("저거 보스면 패턴 보고 들어가는게 나아")

In [ ]:
#91
await invoke_graphs("나 좀 약해보이지 않아?")

In [ ]:
#92
await invoke_graphs("지금 상황 좀 빡센데")

In [ ]:
#93
await invoke_graphs("나 잘하고 있는거 맞지")

In [ ]:
#94
await invoke_graphs("이거 생각보다 무서운데")

In [ ]:
#95
await invoke_graphs("분위기 왜 이래")

In [ ]:
#96
await invoke_graphs("좀 긴장된다")

In [ ]:
#97
await invoke_graphs("여기 느낌 안좋아")

In [ ]:
#98
await invoke_graphs("열받네")

In [ ]:
#99
await invoke_graphs("나 실수한거 같아")

In [ ]:
#100
await invoke_graphs("지금 괜찮은 선택이었어")